<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/DLT_Project_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [6]:
import os
dir_path = '/content/drive/MyDrive/finqa_dataset'
os.chdir(dir_path)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from ashish_test import utils
from utils import *
import json
import pandas as pd
import torch
# from torch.utils.data import Dataset, DataLoader
import time
import evaluate
from datasets import Dataset, load_dataset
import random

In [5]:
# dir_path = '/content/drive/MyDrive/finqa_dataset'
# os.chdir(dir_path)

In [7]:
with open(f"{dir_path}/train.json") as input_file:
        train_data = json.load(input_file)

with open(f"{dir_path}/dev.json") as input_file:
        valid_data = json.load(input_file)

with open(f"{dir_path}/test.json") as input_file:
        test_data = json.load(input_file)

In [8]:
assert len(train_data) == 6251, "The train data seems to be off, please check"
assert len(valid_data) == 883, "The validation data seems to be off, please check"
assert len(test_data) == 1147, "The test data seems to be off, please check"

Loading Ground Truth Serialized Dataset

In [9]:
df_serialized=pd.read_csv(f"{dir_path}/df_combined_table_ser_sample3000.csv")

In [10]:
df_serialized.columns

Index(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'id',
       'dataset', 'prompt', 'serialized_text_gpt35'],
      dtype='object')

In [11]:
df_serialized.shape

(3000, 9)

In [12]:
train_data=pd.DataFrame(train_data).merge(df_serialized[df_serialized['dataset']=='train'][['id','serialized_text_gpt35']],on='id',how='inner').to_dict(orient='records')

In [13]:
train_data[0].keys()

dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all', 'serialized_text_gpt35'])

In [14]:
train_data[0]['qa']

{'question': 'for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?',
 'answer': '7.4',
 'explanation': '',
 'ann_table_rows': [],
 'ann_text_rows': [0, 1],
 'steps': [{'op': 'add1-1', 'arg1': '75.0', 'arg2': '72.7', 'res': '147.7'},
  {'op': 'divide1-2', 'arg1': '#0', 'arg2': '20', 'res': '7.4'}],
 'program': 'add(75.0, 72.7), divide(#0, 20)',
 'gold_inds': {'text_0': 'american tower corporation and subsidiaries notes to consolidated financial statements ( 3 ) consists of customer-related intangibles of approximately $ 75.0 million and network location intangibles of approximately $ 72.7 million .',
  'text_1': 'the customer-related intangibles and network location intangibles are being amortized on a straight-line basis over periods of up to 20 years .'},
 'exe_ans': 7.385,
 'tfidftopn': {'text_8': 'the customer-related intangibles and network location intangibles are being amortized on a straight-line basis 

In [15]:
def prepare_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        id=example["id"]
        table_ori=example["table_ori"]
        answer_steps=example["qa"]["steps"]
        serialized_text=example["serialized_text_gpt35"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"id":id, "context": serialized_text, "question": question, "answer": steps_text,\
                      "table":table_ori,"answer_steps":answer_steps}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

In [16]:
## Preparing huggingface dataset
data_splits = {'train': train_data}
datasets = {split: Dataset.from_pandas(pd.DataFrame(prepare_data(data, False))) for split, data in data_splits.items()}

Net Errors: 98


In [17]:
index=0
datasets['train'][index]

{'id': 'AMT/2012/page_121.pdf-1',
 'context': 'The table represents the preliminary purchase price allocation for an acquisition. The acquisition resulted in the acquisition of current assets amounting to $8,763 and non-current assets amounting to $2,332. The non-current assets included property and equipment worth $26,711 and intangible assets worth $21,079. Other non-current liabilities amounting to ($1,349) were also acquired. The fair value of net assets acquired was $57,536, and the goodwill resulting from the acquisition was $5,998.',
 'question': 'for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?',
 'answer': 'Step 1: Add 75.0 and 72.7. This gives the result: 147.7 ####### Step 2: Divide 147.7 by 20. This gives the result: 7.4',
 'table': [['', 'Preliminary Purchase Price Allocation'],
  ['Current assets', '$8,763'],
  ['Non-current assets', '2,332'],
  ['Property and equipment', '26,711'],
  ['Intan

In [18]:
step_pattern = r"Step \d+"
import re

In [19]:
def num_steps_dict_fn(example):
    example["num_steps"] = len(re.findall(step_pattern, example["answer"]))
    return example

In [20]:
datasets['train']=datasets['train'].map(num_steps_dict_fn)
# datasets['valid']=datasets['valid'].map(num_steps_dict_fn)
# datasets['test']=datasets['test'].map(num_steps_dict_fn)

Map:   0%|          | 0/2902 [00:00<?, ? examples/s]

In [21]:
datasets['train']

Dataset({
    features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
    num_rows: 2902
})

**Filtering Datasets for 1 Step**

In [22]:
datasets['train']=datasets['train'].filter(lambda example: example['num_steps']==1)
# datasets['valid']=datasets['valid'].filter(lambda example: example['num_steps']==1)
# datasets['test']=datasets['test'].filter(lambda example: example['num_steps']==1)

Filter:   0%|          | 0/2902 [00:00<?, ? examples/s]

In [23]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
     num_rows: 1713
 })}

Loading Model

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [25]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token: ··········


In [26]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",use_auth_token=hf_token)

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [27]:
datasets['train'][15]['answer'],datasets['train'][45]['answer'],datasets['train'][21]['answer'],datasets['train'][129]['answer']

('Step 1: Add 286 and 272. This gives the result: 558',
 'Step 1: Divide 48 by 116. This gives the result: 41.4%',
 'Step 1: Multiply 0.0425 by 1000. This gives the result: 42.5',
 'Step 1: Subtract 41 from 117. This gives the result: 76')

In [28]:
datasets['train'][15]['question'],datasets['train'][45]['question'],datasets['train'][21]['question'],datasets['train'][129]['question']

('for 2013 and 2012 , what was total noninterest income in millions?',
 'what is the tax benefit as a percentage of nonoperating income ( expense ) on a gaap basis in 2013?',
 'if you held 1000 shares on may 30 , 2014 , how much would you receive in dividends?',
 'the pretax pension expenses that reduced primarily due to the amortization impact of the favorable 2009 investment returns as compared with the expected long-term return assumption resulted in how much of a decreased expense from 2009 to 2010 , in millions?')

In [29]:
datasets['train'][15]['context'],datasets['train'][45]['context'],datasets['train'][21]['context'],datasets['train'][129]['context']

('In the year ended December 31, 2013, the net interest income for the company was $31 million, which decreased from $38 million in 2012. The noninterest income increased from $272 million in 2012 to $286 million in 2013. The total customer-related trading revenue was $317 million in both years. However, the securities underwriting and trading revenue decreased from $100 million in 2012 to $78 million in 2013. The company also generated $94 million and $92 million from foreign exchange in 2013 and 2012, respectively. Additionally, they earned $145 million and $118 million from financial derivatives and other sources in 2013 and 2012, respectively.',
 "In 2013, the nonoperating income on a GAAP basis was $116 million, whereas in 2012, it was a loss of $54 million. In 2011, it was a loss of $114 million. After subtracting the net income attributable to noncontrolling interests, the nonoperating income for 2013 was $97 million, a decrease from 2012's $36 million loss and 2011's $116 milli

In [30]:
datasets['train'][129]

{'id': 'PNC/2009/page_68.pdf-1',
 'context': 'The table lists the estimated increase to the 2010 pension expense in millions of dollars for various changes in assumption. A .5% decrease in the discount rate is estimated to result in a $10 million increase, while a .5% decrease in expected long-term return on assets is estimated to result in an $18 million increase. Additionally, a .5% increase in compensation rate is estimated to result in a $3 million increase to the 2010 pension expense.',
 'question': 'the pretax pension expenses that reduced primarily due to the amortization impact of the favorable 2009 investment returns as compared with the expected long-term return assumption resulted in how much of a decreased expense from 2009 to 2010 , in millions?',
 'answer': 'Step 1: Subtract 41 from 117. This gives the result: 76',
 'table': [['Change in Assumption(a)',
   'EstimatedIncrease to 2010PensionExpense(Inmillions)'],
  ['.5% decrease in discount rate', '$10'],
  ['.5% decrease 

In [31]:
import warnings
warnings.filterwarnings('ignore')
# index = 10 (question not understood clearly), 11 (answer and context irrelevant to the question being asked)
list_indices=[15,45,21,13]

prompt=''
dash_line = '-'.join('' for x in range(100))
for i,j in enumerate(list_indices):
  context = datasets['train'][j]['context']
  answer = datasets['train'][j]['answer']
  question = datasets['train'][j]['question']

  prompt+= f"""
  ### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.

  ### Context:
  {context}

  ### Question:
  {question}

  ### Answer:
  {answer}
  """
  prompt+=dash_line

INDEX_TEST=108
context_inf=datasets['train'][INDEX_TEST]['context']
question_inf=datasets['train'][INDEX_TEST]['question']
answer_inf=datasets['train'][INDEX_TEST]['answer']

prompt+= f"""
### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.
Follow the examples presented above and answer the question in a similar way.

### Context:
{context_inf}

### Question:
{question_inf}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.1,
        do_sample=True,
        max_new_tokens=128,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
# print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
print(f'BASELINE ANSWER:\n{answer_inf}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - FEW SHOT:\n{output}')
print('###ANSWER ABOVE###')

---------------------------------------------------------------------------------------------------
BASELINE ANSWER:
Step 1: Divide 110494 by 9038137. This gives the result: 1%

---------------------------------------------------------------------------------------------------
MODEL GENERATED ANSWER - FEW SHOT:

  ### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.

  ### Context:
  In the year ended December 31, 2013, the net interest income for the company was $31 million, which decreased from $38 million in 2012. The noninterest income increased from $272 million in 2012 to $286 million in 2013. The total customer-related trading revenue was $317 million in both years. However, the securities underwriting and trading revenue decreased from $100 million in 2012 to $78 million in 2013. The company also generated $94 million and $92 million from foreign exchange in 2013 and 2012, r

Post Processing

In [31]:
import re

def extract_info(input_text):
  try:
      # Regular expression to match the answer pattern
      answer_pattern = r"Step 1: ([A-Za-z]+) ([-]?[$]?[\d,.]+) .* ([-]?[$]?[\d,.]+)\.* This gives the result: ([-]?[$]?[\d.]+%?)"

      # Find the matching groups
      match = re.search(answer_pattern, input_text)
      if match:
          operation, arg1, arg2, result = match.groups()
          # Removing commas from the arguments and converting to appropriate types

          chars_to_replace=[',','$']
          for char in chars_to_replace:
              arg1=arg1.replace(char,'')
              arg2=arg2.replace(char,'')
              result=result.replace(char,'')
          # arg1 = float(arg1)
          # arg2 = float(arg2)
          # Checking if result is a percentage or a number
          if '%' in result:
              result = (result.replace('%', ''))
          else:
              result = (result)

          # Constructing the output dictionary
          output = {
              'Answer': match.group(0),
              'Operation': operation.capitalize(),
              'argument_1': arg1,
              'argument_2': arg2,
              'result': result
          }
          return output
  except:
      return "Pattern not found"

In [33]:
split_text = re.split(r'-{5,}', output)
last_segment = split_text[-1].strip()
ans_dict=extract_info(last_segment)

In [34]:
ans_dict

{'Answer': 'Step 1: Divide 110,494 by 9,038,137. This gives the result: 12.1%',
 'Operation': 'Divide',
 'argument_1': '110494',
 'argument_2': '9038137.',
 'result': '12.1'}

In [287]:
ops_dict={'Divide':'/', 'Multiply':'*', 'Add':'+', 'Subtract':'-'}

In [281]:
%%capture
!pip install langchain-experimental
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

In [290]:
ans_dict['argument_1'].replace('.','')+(ops_dict[ans_dict['Operation']])+ans_dict['argument_2'].replace('.','')

'110494/9038137'

In [291]:
value_to_compute=ans_dict['argument_1'].replace('.','')+(ops_dict[ans_dict['Operation']])+ans_dict['argument_2'].replace('.','')
python_repl.run(f"print({value_to_compute})")

'0.012225307051663413\n'

Function to apply on all rows of the Dataset

In [32]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
     num_rows: 1713
 })}

In [33]:
df_train=pd.DataFrame(datasets['train'])

In [34]:
df_train

,id,context,question,answer,table,answer_steps,num_steps
0,AAL/2015/page_74.pdf-1,The table above provides a breakdown of reorga...,what is the percent of the labor-related deeme...,Step 1: Divide 1733 by 2655. This gives the re...,"[[, 2013], [Labor-related deemed claim (1), $1...","[{'arg1': '1733', 'arg2': '2655', 'op': 'divid...",1
1,GS/2014/page_47.pdf-3,The table above shows the financial informatio...,in 2013 what percentage of total net revenues ...,Step 1: Divide 1947 by 7018. This gives the re...,"[[, Year Ended December], [<i>$ in millions</i...","[{'arg1': '1947', 'arg2': '7018', 'op': 'divid...",1
2,AAL/2014/page_89.pdf-3,"In 2013, a total of $2,640 was spent on reorga...",what percentage of total reorganization items ...,Step 1: Divide 1733 by 2640. This gives the re...,"[[, 2013], [Labor-related deemed claim (1), $1...","[{'arg1': '1733', 'arg2': '2640', 'op': 'divid...",1
3,WRK/2018/page_56.pdf-3,The table provides information on payments due...,what was the percent of the total long-term de...,Step 1: Divide 726.6 by 6039.0. This gives the...,"[[, Payments Due by Period], [(In millions), T...","[{'arg1': '726.6', 'arg2': '6039.0', 'op': 'di...",1
4,INTC/2013/page_71.pdf-2,"As of December 28, 2013, the company had a tot...",as part of the proceeds from the clear wire tr...,Step 1: Divide 439 by 470. This gives the resu...,"[[(In Millions), Dec 28,2013, Dec 29,2012], [A...","[{'arg1': '439', 'arg2': '470', 'op': 'divide2...",1
...,...,...,...,...,...,...,...
1708,UNP/2009/page_83.pdf-4,The table displays the amounts of various type...,what was the change in equipment rents payable...,Step 1: Subtract 93 from 89. This gives the re...,"[[<i>Millions of Dollars</i>, <i>Dec. 31, 2009...","[{'arg1': '89', 'arg2': '93', 'op': 'minus2-1'...",1
1709,ABMD/2015/page_86.pdf-3,The table shows the total fair value of an acq...,what portion of total consideration transferre...,Step 1: Divide 15750 by 21750. This gives the ...,"[[, Total Acquisition Date Fair Value (in thou...","[{'arg1': '15750', 'arg2': '21750', 'op': 'div...",1
1710,IP/2012/page_56.pdf-4,"In 2012, the sales of the company were $6,230 ...",what was the operating margin from printing pa...,Step 1: Divide 599 by 6230. This gives the res...,"[[In millions, 2012, 2011, 2010], [Sales, $6,2...","[{'arg1': '599', 'arg2': '6230', 'op': 'divide...",1
1711,BDX/2016/page_62.pdf-3,This table provides information on the average...,what is the mathematical range of dilutive sha...,Step 1: Subtract 4410 from 4972. This gives th...,"[[, 2016, 2015, 2014], [Average common shares ...","[{'arg1': '4972', 'arg2': '4410', 'op': 'minus...",1


In [35]:
list_indices=[15,45,21]

prompt=''
dash_line = '-'.join('' for x in range(100))
for i,j in enumerate(list_indices):
  context = datasets['train'][j]['context']
  answer = datasets['train'][j]['answer']
  question = datasets['train'][j]['question']

  prompt+= f"""
  ### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.

  ### Context:
  {context}

  ### Question:
  {question}

  ### Answer:
  {answer}
  """
  prompt+=dash_line

In [36]:
print(prompt)


  ### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.

  ### Context:
  In the year ended December 31, 2013, the net interest income for the company was $31 million, which decreased from $38 million in 2012. The noninterest income increased from $272 million in 2012 to $286 million in 2013. The total customer-related trading revenue was $317 million in both years. However, the securities underwriting and trading revenue decreased from $100 million in 2012 to $78 million in 2013. The company also generated $94 million and $92 million from foreign exchange in 2013 and 2012, respectively. Additionally, they earned $145 million and $118 million from financial derivatives and other sources in 2013 and 2012, respectively.

  ### Question:
  for 2013 and 2012 , what was total noninterest income in millions?

  ### Answer:
  Step 1: Add 286 and 272. This gives the result: 558
  ----------

In [37]:
def output_fewshot(row,prompt=prompt):
  context = row['context']
  question = row['question']
  answer = row['answer']

  prompt+= f"""
### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.
Follow the examples presented above and answer the question in a similar way.

### Context:
{context}

### Question:
{question}

### Answer:
"""
  try:
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            temperature=0.1,
            do_sample=True,
            max_new_tokens=128,
        )[0],
        skip_special_tokens=True
    )
    ##Post-Processing the Output
    split_text = re.split(r'-{5,}', output)
    last_segment = split_text[-1].strip()
    ans_dict=extract_info(last_segment)

    return ans_dict
  except:
    return {'answer':None}

In [38]:
from tqdm import tqdm
tqdm.pandas(desc="Progress:")

In [41]:
import warnings
warnings.filterwarnings('ignore')
df_train.head(2).progress_apply(lambda row:output_fewshot(row,prompt),axis=1)

Progress:: 100%|██████████| 2/2 [00:34<00:00, 17.29s/it]


0    {'Answer': 'Step 1: Divide 1733 by 2655. This ...
1                                                 None
dtype: object

In [42]:
df_train_subset=df_train.head(500)

In [43]:
df_train_subset.loc[:,"fewshot_answers"]=df_train_subset.progress_apply(lambda row:output_fewshot(row,prompt),axis=1)

Progress:: 100%|██████████| 500/500 [1:58:56<00:00, 14.27s/it]


In [47]:
df_train_subset.to_csv("./fewshot_answers_train_subset.csv",index=False)

In [50]:
df_train_subset[df_train_subset['fewshot_answers'].isna()].loc[1.'context']

,id,context,question,answer,table,answer_steps,num_steps,fewshot_answers
1,GS/2014/page_47.pdf-3,The table above shows the financial informatio...,in 2013 what percentage of total net revenues ...,Step 1: Divide 1947 by 7018. This gives the re...,"[[, Year Ended December], [<i>$ in millions</i...","[{'arg1': '1947', 'arg2': '7018', 'op': 'divid...",1,None
7,HII/2018/page_64.pdf-4,The table above shows the backlog of three dif...,what portion of total backlog is related to in...,Step 1: Divide 7991 by 21367. This gives the r...,"[[, December 31, 2018, December 31, 2017], [($...","[{'arg1': '7991', 'arg2': '21367', 'op': 'divi...",1,None
12,JPM/2018/page_73.pdf-5,The table presents the stock prices and perfor...,did jpmorgan chase outperform the kbw bank index?,Step 1: Check if 189.57 is greater than 137.82...,"[[December 31,(in dollars), 2013, 2014, 2015, ...","[{'arg1': '189.57', 'arg2': '137.82', 'op': 'c...",1,None
19,WRK/2018/page_86.pdf-3,The table provides information on the Weighted...,how much of the cost of the acquisition was no...,Step 1: Subtract 24.8 from 34.6. This gives th...,"[[, Weighted Avg.Life, AmountsRecognized as of...","[{'arg1': '34.6', 'arg2': '24.8', 'op': 'minus...",1,None
20,OKE/2008/page_38.pdf-2,The table shows information about three differ...,"as of january 31 , 2009 what percentage of emp...",Step 1: Divide 312 by 4742. This gives the res...,"[[Union, Employees, Contract Expires], [United...","[{'arg1': '312', 'arg2': '4742', 'op': 'divide...",1,None
...,...,...,...,...,...,...,...,...
477,JPM/2010/page_59.pdf-2,The table provides a breakdown of revenue for ...,what was noninterest revenue as a percent of t...,Step 1: Divide 51693 by 102694. This gives the...,"[[Year ended December 31, (in millions), 2010,...","[{'arg1': '51693', 'arg2': '102694', 'op': 'di...",1,None
484,AAPL/2008/page_38.pdf-2,The given table provides the stock prices of A...,what was the change in cumulative total return...,Step 1: Subtract 114 from const_100. This give...,"[[, Sep-03, Sep-04, Sep-05, Sep-06, Sep-07, Se...","[{'arg1': 'const_100', 'arg2': '114', 'op': 'm...",1,None
487,AAPL/2012/page_38.pdf-1,The table above shows financial data for a com...,what was the increase in annual operating cash...,Step 1: Subtract 18595 from 50856. This gives ...,"[[, 2012, 2011, 2010], [Cash, cash equivalents...","[{'arg1': '50856', 'arg2': '18595', 'op': 'min...",1,None
495,CDNS/2006/page_42.pdf-4,The table displays the allocation of expenses ...,what was the change in research and developmen...,Step 1: Subtract 29% from 31%. This gives the ...,"[[, 2006, 2005, 2004], [Marketing and sales, 2...","[{'arg1': '31%', 'arg2': '29%', 'op': 'minus2-...",1,None


In [56]:
df_train_subset[df_train_subset['fewshot_answers'].isna()].loc[487,'question']

'what was the increase in annual operating cash flow between 2010 and 2012?'

In [57]:
df_train_subset[df_train_subset['fewshot_answers'].isna()].loc[487,'table']

[['', '2012', '2011', '2010'],
 ['Cash, cash equivalents and marketable securities',
  '$121,251',
  '$81,570',
  '$51,011'],
 ['Accounts receivable, net', '$10,930', '$5,369', '$5,510'],
 ['Inventories', '$791', '$776', '$1,051'],
 ['Working capital', '$19,111', '$17,018', '$20,956'],
 ['Annual operating cash flow', '$50,856', '$37,529', '$18,595']]

In [58]:
df_train_subset[df_train_subset['fewshot_answers'].isna()].loc[487,'context']

"The table above shows financial data for a company for the years 2012, 2011, and 2010. The first row lists the different types of assets held by the company, including cash, cash equivalents, and marketable securities. The second row shows the value of these assets in millions of dollars for each year. \n\nThe next few rows show data on accounts receivable and inventories, also in millions of dollars. The final row shows the company's working capital, which is the difference between its current assets and liabilities. \n\nThe last row of the table shows the company's annual operating cash flow, which is the amount of cash generated by its operations each year. This figure is also shown in millions of dollars. \n\nOverall, the table provides a snapshot of the company's financial health over a three-year period, with data on its assets, liabilities, and cash flow."

**Common Issues**


*   Serialization dropped numbers present in the table
*   FinQA's answer itself is flawed



In [48]:
print(prompt)


  ### Instruction: Answer the question in a single step by extracting relevant values from the context given below. Perform the necessary arithmetic operation.

  ### Context:
  In the year ended December 31, 2013, the net interest income for the company was $31 million, which decreased from $38 million in 2012. The noninterest income increased from $272 million in 2012 to $286 million in 2013. The total customer-related trading revenue was $317 million in both years. However, the securities underwriting and trading revenue decreased from $100 million in 2012 to $78 million in 2013. The company also generated $94 million and $92 million from foreign exchange in 2013 and 2012, respectively. Additionally, they earned $145 million and $118 million from financial derivatives and other sources in 2013 and 2012, respectively.

  ### Question:
  for 2013 and 2012 , what was total noninterest income in millions?

  ### Answer:
  Step 1: Add 286 and 272. This gives the result: 558
  ----------